In [5]:
import cv2
import face_recognition
import numpy as np
import uuid
import time


tolerance = 0.45 
detection_interval = 5  


known_users = {}  
active_users = {}  
user_counter = 1
frame_count = 0

video = cv2.VideoCapture(0)

last_scaled_locations = []
last_user_ids = []

while True:
    ret, original_frame = video.read()
    if not ret:
        break

    small_frame = cv2.resize(original_frame, (0, 0), fx=0.25, fy=0.25)
    rgb_small_frame = cv2.cvtColor(small_frame, cv2.COLOR_BGR2RGB)
    rgb_frame = cv2.cvtColor(original_frame, cv2.COLOR_BGR2RGB)

    if frame_count % detection_interval == 0:
        face_locations = face_recognition.face_locations(rgb_small_frame)
        scaled_locations = [(top * 4, right * 4, bottom * 4, left * 4) for (top, right, bottom, left) in face_locations]
        face_encodings = face_recognition.face_encodings(rgb_frame, scaled_locations)

        current_ids = []

        for encoding in face_encodings:
            match_found = False
            best_match_id = None
            best_distance = 1.0

            for uid, known_encoding in known_users.items():
                distance = np.linalg.norm(known_encoding - encoding)
                if distance < tolerance and distance < best_distance:
                    best_distance = distance
                    best_match_id = uid
                    match_found = True

            if match_found:
                current_ids.append(best_match_id)
                active_users[best_match_id] = encoding
            else:
                new_id = f"user_{user_counter}"
                known_users[new_id] = encoding
                active_users[new_id] = encoding
                current_ids.append(new_id)
                user_counter += 1

        last_scaled_locations = scaled_locations
        last_user_ids = current_ids

    for (top, right, bottom, left), user_id in zip(last_scaled_locations, last_user_ids):
        cv2.rectangle(original_frame, (left, top), (right, bottom), (0, 255, 0), 2)
        cv2.putText(original_frame, user_id, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

    cv2.putText(original_frame, f"People Tracked: {len(active_users)}", (10, 30), cv2.FONT_HERSHEY_DUPLEX, 1, (255, 255, 255), 2)

    frame_count += 1
    cv2.imshow("Persistent Face Tracker", original_frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
video.release()
cv2.destroyAllWindows()

# Cleanup
known_users.clear()
active_users.clear()